In [ ]:
##importar paquetes
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
#importar datos 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cargar bases
bd1 = pd.read_csv ('/content/drive/MyDrive/ANALITICA3/employee_survey_data.csv')
bd2 = pd.read_csv ('/content/drive/MyDrive/ANALITICA3/general_data.csv', sep=";")
bd3 = pd.read_csv ('/content/drive/MyDrive/ANALITICA3/in_time.csv', sep=";")
bd4 = pd.read_csv ('/content/drive/MyDrive/ANALITICA3/manager_survey_data.csv', sep=";" )
bd5 = pd.read_csv ('/content/drive/MyDrive/ANALITICA3/out_time.csv', sep=";" )
bd6 = pd.read_csv ('/content/drive/MyDrive/ANALITICA3/retirement_info.csv', sep=";" )

In [ ]:
bd3

**#ANÁLISIS BASES**

SE OBSERVARA SUS COLUMNAS PARA SABER CUALES PODRIAN SER UTILES PARA EL ANÁLISIS Y AQUELLAS QUE NO SIRVAN POR SU NÙMERO DE NULOS EN COMPARACIÓN AL NÙMERA DE DATOS TOTAL QUE TENGAN 

In [ ]:
#ANÁLISIS BASES

lista= [bd1, bd2, bd3, bd4, bd5, bd6]

for i in lista:
  print(i.describe())
  print("NULOS:")
  print(i.isnull().sum())
  print('-------------------------------------------------')


In [ ]:
#SE TRATA LOS NULOS DE CADA BASE
#NO SE TOMARA LAS BASES 3 Y 5 DEBIDO A LA CANTIDAD DE NULOS EXISTENTES Y QUE LA FECHA DE CUANDO SE RETIRARON ESTÁ EN LABASE DE DATOS RETIRADOS
#Y EN LA GENERAL SE TIENENE LOS AÑOS QUE CADA EMPLEADO TRABAJA EN LA EMPRESA POR LO QUE NO SE VE NECESARIA UTILIZARLAS
#BASE1
nulos = ['EnvironmentSatisfaction',  'JobSatisfaction',  'WorkLifeBalance']
for i in nulos:
  bd1[i] = bd1[i].fillna(bd1[i].median())
#BASE2
bd2['NumCompaniesWorked'] = bd2['NumCompaniesWorked'].fillna(bd2['NumCompaniesWorked'].interpolate())
#BASE6
bd6['resignationReason'] = bd6['resignationReason'].fillna('Others')

In [ ]:
#se unen las bases de datos 1, 2, 4 y 6 las otas no 
from operator import concat
bdt= bd1.merge(bd2, on = 'EmployeeID' , how = 'left')
bdto= bdt.merge(bd4, on = 'EmployeeID' , how = 'left')
bdtotal = bdto.merge(bd6, on = 'EmployeeID' , how = 'left')
bdtotal.head(5)


In [ ]:
#Se analizan los tipos de cada columna
print(bdtotal.shape)
print('-------------------------------------------------')
print(bdtotal.isnull().sum())
print('-------------------------------------------------')
print(bdtotal.dtypes)
print('-------------------------------------------------')

(4410, 32)
-------------------------------------------------
EmployeeID                    0
EnvironmentSatisfaction       0
JobSatisfaction               0
WorkLifeBalance               0
Age                           0
BusinessTravel                0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
EmployeeCount                 0
Gender                        0
JobLevel                      0
JobRole                       0
MaritalStatus                 0
MonthlyIncome                 0
NumCompaniesWorked            0
Over18                        0
PercentSalaryHike             0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             9
TrainingTimesLastYear         0
YearsAtCompany                0
YearsSinceLastPromotion       0
YearsWithCurrManager          0
JobInvolvement                0
PerformanceRating             0
Attrition                  3699
retirementD

In [ ]:
#Se arregla los tipos de datos de las columnas
bdtotal=bdtotal.convert_dtypes()
bdtotal.dtypes

In [ ]:
#Se mira si la columna esta en formato fecha y se le arregla el tipo de dato
print(bdtotal[['retirementDate']].head(10))
bdtotal['retirementDate']= pd.to_datetime(bdtotal['retirementDate'], errors= 'coerce', format='%Y-%m-%d')
bdtotal.dtypes

Debido a que las columnas :Attrition, retirementDate, retirementType, resignationReason, tienen nulos porque la base de datos retiros solo tiene 711 registros, al solo tomar en cuenta los empleados que se han retirado y no el nùmero total. Por lo que se decide reemplazar estos nulos por "NO RETIRADOS"

In [ ]:
#Reemplazo de nulos
bdtotal['Attrition'] = bdtotal['Attrition'].fillna('Vigente')
bdtotal['retirementDate'] = bdtotal['retirementDate'].fillna('No aplica')
bdtotal['retirementType'] = bdtotal['retirementType'].fillna('No aplica')
bdtotal['resignationReason'] = bdtotal['resignationReason'].fillna('No aplica')
bdtotal.head(5)

In [ ]:
#Cambiar el nombre de las columnas a minusculas
bdtotal.columns = bdtotal.columns.str.lower()
bdtotal.columns

In [ ]:
#Homologación de categorias 
listaf= ['environmentsatisfaction', 'jobsatisfaction',
        'worklifebalance', 'age', 'businesstravel', 'department',
        'distancefromhome', 'education', 'educationfield', 'employeecount',
        'gender','joblevel', 'jobrole', 'maritalstatus', 'monthlyincome',
        'numcompaniesworked', 'over18', 'percentsalaryhike', 'standardhours',
        'stockoptionlevel', 'totalworkingyears', 'trainingtimeslastyear',
        'yearsatcompany', 'yearssincelastpromotion', 'yearswithcurrmanager',
        'jobinvolvement', 'performancerating', 'attrition', 'retirementdate',
        'retirementtype', 'resignationreason']

In [ ]:
for i in listaf:
  print(bdtotal[i].value_counts())
  print('-----------------------------------------------')

3.0    1375
4.0    1334
2.0     856
1.0     845
Name: environmentsatisfaction, dtype: int64
-----------------------------------------------
4.0    1367
3.0    1343
1.0     860
2.0     840
Name: jobsatisfaction, dtype: int64
-----------------------------------------------
3.0    2698
2.0    1019
4.0     454
1.0     239
Name: worklifebalance, dtype: int64
-----------------------------------------------
35    234
34    231
31    207
36    207
29    204
32    183
30    180
38    174
33    174
40    171
37    150
28    144
27    144
42    138
39    126
45    123
41    120
26    117
46     99
44     99
43     96
50     90
24     78
25     78
49     72
47     72
55     66
48     57
51     57
53     57
52     54
54     54
22     48
58     42
23     42
56     42
21     39
20     33
59     30
19     27
18     24
60     15
57     12
Name: age, dtype: int64
-----------------------------------------------
Travel_Rarely        3129
Travel_Frequently     831
Non-Travel            450
Name: businesstr

**HOMOLOGACIÓN DE CATEGORIAS**

SE CONCLUYE QUE LAS COLUMNAS AGE, DISTANCEKM, MONTHLYINCOME, NUMCOMPANIESWORKED, PERCENTSALARY, TOTALWORKINGYEARS, YEARSATCOMPANY, YEARSSINCELASTPROMOTION Y YEARSWITHCURRMANAGER DEBEN SER AGRUPADAS POR INTERVALOS DEBIDO A SU AMPLIA CANTIDAD DE CATEGORIAS.

ADEMÁS SE ELIMINARA EMPLOYEECOUNT, OVER18 Y STANDAR HOURS DEBIDO A QUE SOLO POSEEN UNA CATEGORIA


In [ ]:
#Eliminar columnas
bdtotal1=bdtotal.drop(['employeecount','over18','standardhours'], axis=1)

In [ ]:
#AGRUPAR CATEGORIAS


In [ ]:
#Homologación de categorias de tipo de dato integer
lista1= ['businesstravel', 'department' , 'educationfield', 'gender', 'jobrole', 'maritalstatus', 'over18',
'attrition', 'retirementtype', 'resignationreason']
for i in lista1:
  print(bdtotal[i].value_counts())
  print('-----------------------------------------------')

In [ ]:
bdtotal['gender'].value_counts()

Male      2646
Female    1764
Name: gender, dtype: Int64

In [ ]:
#Homologación de categorias de tipo de dato string
lista1= ['businesstravel', 'department' , 'educationfield', 'gender', 'jobrole', 'maritalstatus', 'over18',
'attrition', 'retirementtype', 'resignationreason']
for i in lista1:
  print(bdtotal[i].value_counts())
  print('-----------------------------------------------')


In [ ]:
bdtotal.columns
bdtotal.values
print(bdtotal['EnvironmentSatisfaction'].dtypes)
#value.counts()

In [ ]:
# Homologacion de categorias(ya todas)
bd1['EnvironmentSatisfaction'].unique()
#para sacar los valores unicos de una columna
df2['mpaa_rating'].unique()
#convierto la fecha a un mejor formato
acc['FECHA']=acc['FECHA'].apply(lambda x:str(x)[0:10])#con esto le quito la hora a la fecha

array([ 3.,  2.,  4.,  1., nan])